
```Author: Yao Gahounzo```

This program solves the 2D-diffusion problem using the continuous Galerkin (CG) method. The method is implemented with two different boundary conditions, Dirichlet, and Neumann. The CG method derivation and all the necessary mathematics are in the file [CG_method.pdf](./CG_method.pdf).

This program call the module func_2D which contains all the subroutines needed in the CG method.


In [ ]:
%matplotlib notebook
%pylab
from scipy.interpolate import griddata
from matplotlib import cm
import pandas as pd
from time import perf_counter
from sympy import*
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Call module func_2D
from func_2D import*

In [ ]:
# Domain of integration, look into func_2D module to add your own problem or initial condition

def domain(icase):
    
    
    if(icase == 1):
        ax = -1 ; bx = 1 ; coeff = 1.0
    elif(icase == 2):
        ax = 0 ; bx = 2*pi ; coeff = 1.0
    elif(icase == 3):
        ax = -1 ; bx = 1 ; coeff = 1.0
    elif(icase == 4):
        ax = -1 ; bx = 1 ; coeff = 0.01
    
    print("====================================")
    print("Problem: Diffusion")
    print("Domain: [{}, {}]".format(ax,bx)) 
    print("Diffusivity: {}".format(coeff))
    #print("====================================\n")
    
    return ax, bx, coeff

# Boundary conditions
def bc_type(alpha,beta):
    
    if(alpha == 0 and beta == 1):
        print("Boundary conditions: Dirichlet")
    elif(alpha == 1 and beta == 0):
        print("Boundary conditions: Neumann")
    print("====================================\n")    

In [ ]:
order = array([4])        # polynomial order
N_element = array([4])
kstages = 3
#cfl = 0.25
dt = 1e-2
Tfinal = 0.5

time_method = "BDF3"      # IRK or BDF3
integration_type = 1      # % = 1 is inexact and = 2 is exact
iplot = False             # plot the solution
icase = 4                 # select icase: 1,2,3,4

alpha = 0              
beta = 1                  # Neumann: alpha = 1, beta = 0
                          # Dirichlet: alpha = 0, beta = 1
    
# Domain and diffusion coefficient

ax,bx,c = domain(icase)
bc_type(alpha,beta)

len_el = len(N_element)
len_pol = len(order)
l2e_norm = zeros((len_pol, len_el))
max_norm = zeros((len_pol, len_el))

Nv = N_element
    
for iN,N in enumerate(order):
    
    cfl = 1/(N+1)       # cfl number
    
    N = order[iN]
    if (integration_type == 1):
        Q = N
    elif (integration_type == 2):
        Q = N+1

    wall = 0
    

    for e, nel in enumerate(Nv):
            
        Nelx = nel; Nely = nel
        Nx = Nelx*N+1
        Ny = Nely*N+1
        Np = Nx*Ny
        Ne = Nelx*Nely
        #Nbound = 2*Nelx + 2*(Nely)
        Nbound = 2*Nx + 2*(Ny-2)
        Nside = 2*Ne + Nelx + Nely
        
        tic = perf_counter()
        
    
        qe, q,coord,intma = diffusion_solver(N,Q,Ne, Np, ax, bx, Nelx, Nely, Nx, Ny, Nbound,Nside,\
                                             icase,Tfinal,dt,c,cfl,kstages,time_method,alpha,beta)
        
        #Compute Norm
        
        top = 0
        bot = 0

        for i in range(Np):
            top += (q[i] - qe[i])**2
            bot += qe[i]**2

        e2 = sqrt(top/bot)
        
        l2e_norm[iN,e] = e2
        

In [ ]:
xmin = min(coord[:,0])
xmax = max(coord[:,0])
ymin = min(coord[:,1])
ymax = max(coord[:,1])
xe = coord[:,0]
ye = coord[:,1]
nx = 200
ny = 200
dx = (xmax-xmin)/nx
dy = (ymax-ymin)/ny
x1 = arange(xmin,xmax+dx,dx)
y1 = arange(ymin,ymax+dy,dy)
xi,yi = meshgrid(x1,y1)

In [ ]:

q_2d = griddata((xe,ye),q,(xi,yi), method='cubic')

fig = figure(2)
fx = fig.add_subplot(111, projection='3d')
surf = fx.plot_surface(xi,yi,q_2d,rstride = 1, cstride = 1,cmap=cm.coolwarm,antialiased=False)
fig.colorbar(surf)

title("Numerical solution")
xlabel("x")
ylabel("y")

show()

In [ ]:

qe_2d = griddata((xe,ye),qe,(xi,yi), method='cubic')

fig = figure(3)
fx = fig.add_subplot(111, projection='3d')
surf = fx.plot_surface(xi,yi,qe_2d,rstride = 1, cstride = 1, cmap=cm.coolwarm,antialiased=False)
fig.colorbar(surf,anchor=(0, 0.3), shrink=0.7)
title("Exact solution")
xlabel("x")
ylabel("y")
show()

In [ ]:
figure(4)
imshow(q_2d, extent=[ax, bx, ax, bx],origin='lower',cmap=cm.coolwarm)
colorbar()
clim(q.min(), q.max())
title("Numerical solution")
xlabel("x")
ylabel("y")
show()

In [ ]:
print("min_q = ",q.min())
print("max_q = ",q.max())